In [5]:
import pyaudio
import speech_recognition as sr
import keyboard

In [6]:
def transcribe_audio(timeout=10, phrase_time_limit=15, mic_index=None):
    recognizer = sr.Recognizer()

    # Use selected mic or default
    mic = sr.Microphone(device_index=mic_index)

    try:
        with mic as source:
            print(" Adjusting for ambient noise...")
            recognizer.adjust_for_ambient_noise(source)
            print("Recording started. Speak now...")

            audio = recognizer.listen(source, timeout=timeout, phrase_time_limit=phrase_time_limit)

            print("Recording ended. Processing...")

        transcript = recognizer.recognize_google(audio)
        print(" Transcribed Text:", transcript)

        with open("transcription.txt", "w") as f:
            f.write(transcript + "\n")

        print(" Transcription saved to file.")
        return transcript

    except sr.WaitTimeoutError:
        print(" No speech detected. Timeout!")
        return None
    except sr.UnknownValueError:
        print(" Couldn't understand audio.")
        return None
    except sr.RequestError as e:
        print(f" API error: {e}")
        return None
    except Exception as e:
        print(f" Unexpected error: {e}")
        return None


In [7]:
def main():
    print("Starting real-time audio detection...")
    while True:
        command = input("Press Enter to check audio or 'q' to quit: ")
        if command.lower() == 'q':
            print("Exiting...")
            break
        transcript = transcribe_audio()

In [16]:
if __name__ == "__main__":
    main()

Starting real-time audio detection...
 Adjusting for ambient noise...
Recording started. Speak now...
Recording ended. Processing...
 Transcribed Text: good morning
 Transcription saved to file.
 Adjusting for ambient noise...
Recording started. Speak now...
Recording ended. Processing...
 Transcribed Text: hello good morning
 Transcription saved to file.
Exiting...


In [14]:
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the tokenizer
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Load the Keras model
model = load_model("call_scam_model.h5")

In [17]:
with open("transcription.txt", "r") as file:
    transcribed_text = file.read().strip()
sequence = tokenizer.texts_to_sequences([transcribed_text])

# Pad the sequence (update maxlen to match your training)
padded = pad_sequences(sequence, maxlen=100)

# Predict
prediction = model.predict(padded)

# Output
if prediction[0][0] >= 0.5:
    print("Scam Detected!")
else:
    print("Genuine Message.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
 Genuine Message.
